In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
df_train = pd.read_csv('../data/train/sentiment_analysis_trainingset.csv')

In [ ]:
df_valid = pd.read_csv('../data/valid/sentiment_analysis_validationset.csv')

In [ ]:
df_testa = pd.read_csv('../data/testa/sentiment_analysis_testa.csv')

In [3]:
df_train

,id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,...,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
0,0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...",-2,-2,-2,-2,1,-2,-2,-2,...,-2,-2,-2,-2,-2,-2,1,-2,1,-2
1,1,"""第三次参加大众点评网霸王餐的活动。这家店给人整体感觉一般。首先环境只能算中等，其次霸王餐提...",-2,-2,-2,-2,-2,-2,-2,0,...,0,0,0,0,1,-2,-2,-2,1,-2
2,2,"""4人同行 点了10个小吃\n榴莲酥 榴莲味道不足 松软 奶味浓\n虾饺 好吃 两颗大虾仁\...",-2,-2,-2,-2,0,-2,1,0,...,-2,-2,1,-2,0,1,-2,-2,0,-2
3,3,"""之前评价了莫名其妙被删 果断继续差评！ 换了菜单 价格更低 开始砸牌子 但套餐还是有150...",-2,-2,-2,-2,-2,-2,-2,0,...,-2,-2,-2,-2,-2,-1,-2,-2,-1,-1
4,4,"""出乎意料地惊艳，椰子鸡清热降火，美容养颜，大大满足了爱吃火锅怕上火星人。椰子冻是帅帅的老板...",-2,-2,-2,-2,-2,-2,-2,-2,...,-2,-2,-2,-2,-2,1,1,-2,1,-2
5,5,"""烤鸭使用的蘸酱很地道，觉得烤鸭好不好吃在很大程度上都取决于这个酱料是否正宗。个人感觉他们家...",-2,-2,-2,-2,1,-2,-2,0,...,-2,-2,1,-2,0,1,-2,-2,1,-2
6,6,"""在好友圈很红的原因很诡异.竟然基本都是我外地的朋友转发的.不远千里跑来厦门买回去吃.吃完各...",-2,-2,-1,-2,-2,-2,-2,-1,...,-2,-2,-1,-2,-1,1,1,1,1,-2
7,7,"""中午饭点人比较多需要排队～2点左右去吃的刚好可以用团购～也不用排队～就是平安夜店员说的都不...",-2,-2,-2,0,1,-2,-2,-2,...,-2,-2,-2,-1,-1,0,-2,1,1,1
8,8,"""跟朋友相约25号看港囧，首映当天排片率超高，为了提前订好票，选了文投国际影城，为了看电影我...",-2,1,-2,-2,-2,-2,-2,1,...,1,-2,-2,-2,-2,1,-2,-2,1,1
9,9,"""首先感谢大众点评再次抽中我免费品尝火锅，同时也感谢古蜀雒味三星堆主题火锅商家提供这个平台，...",-2,-2,1,-2,1,-2,-2,-2,...,1,1,1,1,0,-2,1,-2,0,1


In [5]:
df_train.shape

(105000, 22)

## 查看每个类别占比，数据平衡性问题

In [38]:
data_size = len(df_train)
colnames = df_train.columns[2:]
num_type = len(colnames)

sum_0, sum_1, sum_2, sum_3 = 0, 0, 0, 0
# 查看所有数据的类别分布
for colname in colnames:
    sum_0 += np.sum(df_train[colname] == 1)
    sum_1 += np.sum(df_train[colname] == 0)
    sum_2 += np.sum(df_train[colname] == -1)
    sum_3 += np.sum(df_train[colname] == -2)
print('>>> class distribution in total data:')
print('class  1: {:5.2f}%'.format(sum_0/data_size/num_type*100))
print('class  0: {:5.2f}%'.format(sum_1/data_size/num_type*100))
print('class -1: {:5.2f}%'.format(sum_2/data_size/num_type*100))
print('class -2: {:5.2f}%'.format(sum_3/data_size/num_type*100))
print()

print('============================================================\n')

# 查看每个细粒度中类别的分布
print('>>> class distribution in each type:')
for colname in colnames:
    sum_0 = np.sum(df_train[colname] == 1)
    sum_1 = np.sum(df_train[colname] == 0)
    sum_2 = np.sum(df_train[colname] == -1)
    sum_3 = np.sum(df_train[colname] == -2)
    print('type: {}'.format(colname))
    print('class  1: {:5.2f}%'.format(sum_0/data_size*100))
    print('class  0: {:5.2f}%'.format(sum_1/data_size*100))
    print('class -1: {:5.2f}%'.format(sum_2/data_size*100))
    print('class -2: {:5.2f}%'.format(sum_3/data_size*100))
    print()

>>> class distribution in total data:
class  1: 24.45%
class  0:  8.64%
class -1:  4.30%
class -2: 62.62%


>>> class distribution in each type:
type: location_traffic_convenience
class  1: 20.24%
class  0:  1.00%
class -1:  1.26%
class -2: 77.51%

type: location_distance_from_business_district
class  1: 19.24%
class  0:  0.51%
class -1:  0.56%
class -2: 79.70%

type: location_easy_to_find
class  1: 17.09%
class  0:  2.35%
class -1:  3.79%
class -2: 76.77%

type: service_wait_time
class  1:  4.59%
class  0:  4.17%
class -1:  2.89%
class -2: 88.35%

type: service_waiters_attitude
class  1: 39.40%
class  0: 11.94%
class -1:  8.27%
class -2: 40.39%

type: service_parking_convenience
class  1:  3.76%
class  0:  1.39%
class -1:  1.26%
class -2: 93.60%

type: service_serving_speed
class  1:  8.03%
class  0:  2.27%
class -1:  5.23%
class -2: 84.48%

type: price_level
class  1: 14.82%
class  0: 23.09%
class -1: 11.79%
class -2: 50.30%

type: price_cost_effective
class  1: 17.79%
class  0:  2.9

In [8]:
# 对数量较少的类别样本进行升采样，保证每个类别数量的平衡
data_size = len(df_train)
colnames = df_train.columns[2:]
num_type = len(colnames)

for i in range(data_size):
    pass

array([[-2, -2, -2, ..., -2,  1, -2],
       [-2, -2, -2, ..., -2,  1, -2],
       [-2, -2, -2, ..., -2,  0, -2],
       ...,
       [-2, -2, -2, ...,  0,  0, -2],
       [-2, -2,  1, ..., -2,  1,  1],
       [-2,  1,  0, ..., -2,  0, -2]])

## 转换原始数据为全类别模式

In [ ]:
col_names = df.columns[2:]
df_new = df[df.columns[:2]]
for col_name in col_names:
    df_one_new = pd.get_dummies(df[col_name], prefix=col_name)
    cols = df_one_new.columns
    cols = cols[::-1]
    df_one_new = df_one_new[cols]
    df_new = pd.concat([df_new, df_one_new], axis=1)

In [ ]:
df_new

In [ ]:
df_new.shape

In [ ]:
# write out train dataset
df_new.to_csv(path_or_buf='./data/df_train.csv', header=True, index=False)

In [ ]:
# write out valid dataset
df_new.to_csv(path_or_buf='./data/df_valid.csv', header=True, index=False)

In [ ]:
# write out testa dataset
df_new.to_csv(path_or_buf='./data/df_testa.csv', header=True, index=False)

In [ ]:
df_new['content'].iloc[0]

In [ ]:
df_new[df_new.columns[2:]].shape

In [ ]:
df_new[:2]

In [ ]:
df_testa_pred = pd.read_csv('./data/df_testa_pred.csv')

In [ ]:
df_testa_pred

In [ ]:
df_testa_pred.shape

In [ ]:
df_testa_pred[df_testa_pred.columns[2:]][0:3]

## 查看valid set推测结果

In [ ]:
df_valid = pd.read_csv('./data/df_valid.csv')
df_valid_pred = pd.read_csv('./data/df_valid_pred.csv')

In [ ]:
idx = 2

In [ ]:
print(df_valid['content'].iloc[idx])

In [ ]:
pd.concat([df_valid.iloc[idx, 2:], df_valid_pred.iloc[idx, 2:]], axis=1)

In [ ]:
# 查看真值和推测值每个粒度下类别unique值
for i in range(20):
    print('{:2}: true: {}, pred: {}'.format(i+1, df_valid.iloc[:, i+2].unique(), df_valid_pred.iloc[:, i+2].unique()))

In [ ]:
np.sum(df_valid.iloc[:, 2] == -2)

In [ ]:
np.sum(df_valid_pred.iloc[:, 2] == -2)

## check df_testa_pred

In [ ]:
df_testa_pred = pd.read_csv('./data/df_testa_pred.csv')

In [ ]:
df_testa_pred.shape

In [ ]:
for i in range(20):
    print('{:2}: pred: {}'.format(i+1, df_testa_pred.iloc[:, i+2].unique()))

In [ ]:
idx = 101

In [ ]:
print(df_testa_pred['content'].iloc[idx])

In [ ]:
pd.DataFrame(df_testa_pred.iloc[idx, 2:])